In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/transformer

/content/gdrive/MyDrive/transformer


In [ ]:
!pip install --upgrade datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [5]:
# Importing required libraries
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
import torch

# Define a function to load and prepare the dataset
def load_and_prepare_data(file_path):
    """
    Loads JSON data from the given file path and prepares it with prompts for fine-tuning.
    """
    # Prompt engineering template
    prompt_template = (
        "You are a Transformer-based assistant specializing in helping beginners understand both coding and conceptual questions "
        "related to machine learning and Transformers. Below is a question. Provide a detailed explanation or relevant code example.\n\n"
        "Question:\n{query}\n\n"
        "Answer:"
    )

    # Load the JSON data
    with open(file_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    # Format the data
    formatted_data = [
        {
            "prompt": prompt_template.format(query=entry["content"]),
            "completion": "Provide a comprehensive explanation or solution."
        }
        for entry in raw_data if "content" in entry
    ]

    return Dataset.from_list(formatted_data)

# Load the training and validation datasets
train_dataset = load_and_prepare_data("/content/gdrive/MyDrive/transformer/datasets/train_data.json")
val_dataset = load_and_prepare_data("/content/gdrive/MyDrive/transformer/datasets/val_data.json")

# Define the pretrained model path
pretrained_model_path = "/content/gdrive/MyDrive/transformer/Qwen2.5-Coder-0.5B-Instruct"

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_path, trust_remote_code=True)

# Tokenize the datasets
def tokenize_data(examples):
    """
    Tokenizes the dataset using the provided tokenizer.
    """
    inputs = tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=512)
    outputs = tokenizer(examples["completion"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = outputs["input_ids"]
    return inputs

train_dataset = train_dataset.map(tokenize_data, batched=True)
val_dataset = val_dataset.map(tokenize_data, batched=True)

# Data collator for training
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./tuned_qwen2.5-coder",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    save_total_limit=2,
    push_to_hub=False,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("/content/gdrive/MyDrive/transformer/tuned_qwen2.5-coder")
tokenizer.save_pretrained("/content/gdrive/MyDrive/transformer/tuned_qwen2.5-coder")

print("Fine-tuning complete. Model saved!")


Map:   0%|          | 0/1228 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-f071de9e5983>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.451300,1.469843
2,0.785200,1.541184
3,0.324600,1.774942


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Fine-tuning complete. Model saved!
